https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=ko

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 4.0MB/s 


In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ["accuracy"])
    
    return model

In [6]:
tuner = kt.Hyperband(model_builder,
                      objective = "val_accuracy",
                      max_epochs = 10,
                      factor = 3,
                      directory = "my_dir",
                      project_name = "intro_to_kt")

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [8]:
tuner.search(img_train, label_train, epochs=10, validation_data=(img_test, label_test),
             callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is {best_hps.get("units")} and the optimal learning rate for the optimizer
    is {best_hps.get("learning_rate")}.
""")

Trial 30 Complete [00h 00m 45s]
val_accuracy: 0.8504999876022339

Best val_accuracy So Far: 0.8927000164985657
Total elapsed time: 00h 10m 42s
INFO:tensorflow:Oracle triggered exit

    The hyperparameter search is complete. The optimal number of units in the first densely-connected
    layer is 448 and the optimal learning rate for the optimizer
    is 0.001.



In [9]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5917 - accuracy: 0.7927 - val_loss: 0.4023 - val_accuracy: 0.8578
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3607 - accuracy: 0.8683 - val_loss: 0.3709 - val_accuracy: 0.8653
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3234 - accuracy: 0.8808 - val_loss: 0.3500 - val_accuracy: 0.8741
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2969 - accuracy: 0.8908 - val_loss: 0.3572 - val_accuracy: 0.8725
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2817 - accuracy: 0.8966 - val_loss: 0.3718 - val_accuracy: 0.8708
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2679 - accuracy: 0.8998 - val_loss: 0.3791 - val_accuracy: 0.8613
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2517 - accuracy: 0.9057 - val_loss: 0.3493 - val_accuracy: